# Exploring Ebay Car Sales Data

<span style="font-family:verdana">The aim of this project is to clean the data and analyze the included used car listings. Here we work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.</span>

<span style="font-family:verdana">Let's import some libraries that we need.</span>

In [180]:
import pandas as pd
import numpy as np

## {Step 1} Get the data

In [181]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [182]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We can make some observations like:
- The database contains 20 columns.
- Some columns have null values, but only one has ~20% null values.
- The column names use camelCase istead of Python's preferred snake_case, thats mean that we can't just replace spaces with underscores.

## {Step 2} Data Cleaning

<span style="font-family:verdana">Let's clean the data - to make it easier to work with.</span>

### Cleaning Column Names

<span style="font-family:verdana">We will convert the column names from camelCase to snake_case. Also, we will reword some of the column names based on the data dictionary to be more descriptive.</span>

In [183]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [184]:
autos.rename(columns = {
    "dateCrawled": "date_crawled",
    "offerType": "offer_type",
    "vehicleType": "vehicle_type",
    "yearOfRegistration": "registration_year",
    "powerPS": "power_ps",
    "monthOfRegistration": "registration_month",
    "fuelType": "fuel_type",
    "notRepairedDamage": "unrepaired_damage",
    "dateCreated": "ad_created",
    "nrOfPictures": "num_pictures",
    "postalCode": "postal_code",
    "lastSeen": "last_seen"
}, inplace=True)

In [185]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Initial Data Exploration

<span style="font-family:verdana">Explore the data with using describe() method to find obvious areas where we can begin cleaning the data.</span>

In [186]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


<span style="font-family:verdana">The following observations can be made:
- <i>seller</i> and <i>offer_type</i> have nearly all values same ( freq ),
- <i>num_pictures</i> are 0 for all the entries.

Thats why <i>seller, offer_type</i> and <i>num_pictures</i> can be dropped,

- <i>registration_year</i> has min value 1000 which was long before cars were invented and max value 9999 which is many years in future,
- <i>registration_month</i> has min 0 which is invalid as months range in 1 to 12.</span>

### Dropping columns with mostly one value

In [187]:
autos = autos.drop(["seller", "offer_type", "num_pictures"], axis=1)

In [188]:
autos.shape

(50000, 17)

### Cleaning numerical data

<span style="font-family:verdana">Let's rename column <i>odometer</i> to <i>odometer_km</i> to make more comfortable to undertanding what does this column mean.</span>

In [189]:
autos.rename(columns = {"odometer":"odometer_km"}, inplace=True)
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


#### Exploring <i>price</i> and <i>odometer_km</i>

<span style="font-family:verdana">We explore <i>price</i> and <i>odometer_km</i> to find if the data in these columns need to be cleaned further.</span>

In [190]:
autos["price"].describe()

count     50000
unique     2357
top          $0
freq       1421
Name: price, dtype: object

In [191]:
autos["price"].value_counts().sort_index().head(50)

$0            1421
$1             156
$1,000         639
$1,039           1
$1,040           1
$1,049           6
$1,050          95
$1,059           1
$1,070           1
$1,080           6
$1,090           4
$1,095           3
$1,098           1
$1,099          44
$1,100         376
$1,111          39
$1,112           1
$1,119           1
$1,120           2
$1,149          10
$1,150         226
$1,169           1
$1,170           1
$1,180           4
$1,189           1
$1,190          37
$1,195           1
$1,199         126
$1,200         639
$1,201           2
$1,209           1
$1,212           2
$1,221           1
$1,222           3
$1,234           3
$1,234,566       1
$1,240           2
$1,247           1
$1,249           4
$1,250         335
$1,265           1
$1,270           4
$1,275           1
$1,280          12
$1,285           3
$1,290          38
$1,295           1
$1,299         135
$1,300         371
$1,300,000       1
Name: price, dtype: int64

In [192]:
autos["price"].value_counts().sort_index(ascending=False).head(50)

$999,999         2
$999,990         1
$999           434
$998             5
$996             1
$995             5
$990           147
$99,999,999      1
$99,900          2
$99,000          2
$99             19
$989             2
$985             4
$980            48
$98,500          1
$975             2
$970             7
$965             2
$960             1
$958             1
$951             1
$950           379
$949            11
$945             2
$940             2
$94,999          1
$930             4
$93,911          1
$93,000          2
$925             1
$920             1
$919             1
$910             1
$900           420
$90              5
$9,999         131
$9,998           2
$9,995           5
$9,990          84
$9,989           1
$9,980          10
$9,975           2
$9,970           1
$9,950          32
$9,900         135
$9,899           2
$9,895           2
$9,890          11
$9,870           2
$9,850          22
Name: price, dtype: int64

In [193]:
autos["price"].value_counts().head(50)

$0        1421
$500       781
$1,500     734
$2,500     643
$1,000     639
$1,200     639
$600       531
$800       498
$3,500     498
$2,000     460
$999       434
$750       433
$900       420
$650       419
$850       410
$700       395
$4,500     394
$300       384
$2,200     382
$950       379
$1,100     376
$1,300     371
$3,000     365
$550       356
$1,800     355
$5,500     340
$350       335
$1,250     335
$1,600     327
$1,999     322
$400       321
$6,500     300
$1,400     292
$250       291
$2,800     291
$2,300     290
$1,350     276
$1,700     268
$200       266
$450       265
$3,800     264
$3,200     261
$2,900     256
$1,450     252
$4,000     246
$2,999     242
$2,600     241
$1,900     239
$5,000     239
$1,650     238
Name: price, dtype: int64

<span style="font-family:verdana">As we can see the most values are rounded, which might indicate that the sellers tend to round-up prices. Additionally, 10778 entries are 0 - given that this is only ~3% of the total cars, we might consider removing these. There are a number of listings with prices less than 50, and a few listings with prices more than 1 million. Given  that eBay is an auction site, there could be items where the bid opens at 1. We will however remove items listed above 350000 since the prices seem to increase steadily after that.</span>

In [194]:
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = pd.to_numeric(autos["price"], errors="coerce")
autos = autos[autos["price"].between(1, 350000)]
autos["price"].describe()

C:\Users\Shriksanka\AppData\Local\Temp\ipykernel_19220\3858822736.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  autos["price"] = autos["price"].str.replace("$", "")


count    9939.000000
mean      578.528323
std       268.770245
min         1.000000
25%       350.000000
50%       600.000000
75%       800.000000
max       999.000000
Name: price, dtype: float64

<span style="font-family:verdana">Here is for <i>odometer_km</i>:</span>

In [195]:
autos["odometer_km"] = autos["odometer_km"].str.replace("km", "")
autos["odometer_km"] = autos["odometer_km"].str.replace(",", "")
autos["odometer_km"].describe()

count       9939
unique        13
top       150000
freq        8100
Name: odometer_km, dtype: object

In [196]:
autos["odometer_km"].value_counts().sort_index()

10000       25
100000     255
125000     717
150000    8100
20000       70
30000       32
40000       20
5000       329
50000       26
60000       56
70000       63
80000       81
90000      165
Name: odometer_km, dtype: int64

<span style="font-family:verdana">All values are rounded, which indicates that the sellers might have to select from pre-set options for this field. There are more high mileage cars.</span>

### Dealing with Incorrect Registration Data

<span style="font-family:verdana">As we noticed earlier, there were a few invalid listings of year and month. Since a car can't be put up for sale before being registred and the listings are from 2016(as we saw in step 1), any vehicle with a registration year above 2016 is inaccurate</span>

In [197]:
autos.loc[autos["registration_year"]<1910,"registration_year"]

10556    1800
24511    1111
32585    1800
Name: registration_year, dtype: int64

<span style="font-family:verdana">After seeing the data, it is safe to assume that valid listings have registration years between 1910 and 2016. We remove all other entries.</span>

In [198]:
autos = autos[autos["registration_year"].between(1910, 2016)]
autos["registration_year"].value_counts(normalize=True).sort_values(ascending=False)

1999    0.123799
1997    0.122850
2000    0.121900
1998    0.113984
1996    0.087704
1995    0.082744
2001    0.060264
2016    0.060053
1994    0.037784
2002    0.037045
2003    0.023430
1993    0.021847
1990    0.016675
1992    0.016042
2005    0.015937
1991    0.013931
2004    0.010026
1989    0.005383
2015    0.003272
1988    0.003061
1987    0.002216
2006    0.002005
2007    0.001900
1985    0.001689
1986    0.001689
1980    0.001478
2014    0.001478
2008    0.001161
2009    0.000950
2013    0.000844
2011    0.000633
1970    0.000633
1960    0.000633
2010    0.000528
1910    0.000528
1984    0.000422
2012    0.000422
1972    0.000317
1979    0.000317
1982    0.000317
1964    0.000317
1978    0.000317
1971    0.000211
1981    0.000211
1950    0.000211
1977    0.000211
1967    0.000106
1969    0.000106
1958    0.000106
1966    0.000106
1974    0.000106
1983    0.000106
Name: registration_year, dtype: float64

<span style="font-family:verdana">It turns out that most vehicles were registred in early 2000's. Now we check the <i>registration_month</i>. As mentioned earlier, the min value is 0.</span>

In [199]:
autos["registration_month"].value_counts()

0     1897
3      921
5      718
6      710
4      697
7      653
1      621
12     612
10     588
8      551
9      515
2      509
11     483
Name: registration_month, dtype: int64

<span style="font-family:verdana">Around 30000 entries are 0's. Since there is no way to work with it properly - the most suitable thing will be remove these listings.</span>

In [200]:
autos = autos[~(autos["registration_month"]==0)]
autos["registration_month"].value_counts()

3     921
5     718
6     710
4     697
7     653
1     621
12    612
10    588
8     551
9     515
2     509
11    483
Name: registration_month, dtype: int64

## {Step 3} Exploring

### Exploring the date columns

<span style="font-family:verdana">There are 5 columns that represent date values:</span>
- <span style="font-family:verdana"><i>date_crawled</i>
- <span style="font-family:verdana"><i>ad_created</i>
- <span style="font-family:verdana"><i>last_seen</i>
- <span style="font-family:verdana"><i>registration_year</i>   (already explored)</span>
- <span style="font-family:verdana"><i>registration_month</i> <span style="font-family:verdana">(already explored)</span>
    
<span style="font-family:verdana">Some of these columns were created by the crawler, some came from the website. As of now, <i>date_crawled, last_seen</i> and <i>ad_created</i> are all identified as strings by pandas. </span>

In [201]:
autos[["date_crawled", "ad_created", "last_seen"]].head()

,date_crawled,ad_created,last_seen
6,2016-03-20 17:55:21,2016-03-20 00:00:00,2016-03-23 02:48:59
8,2016-03-22 16:51:34,2016-03-22 00:00:00,2016-03-26 18:18:10
9,2016-03-16 13:47:02,2016-03-16 00:00:00,2016-04-06 10:46:35
16,2016-03-16 14:59:02,2016-03-16 00:00:00,2016-03-18 05:29:37
25,2016-03-21 21:56:18,2016-03-21 00:00:00,2016-04-01 05:16:49


<span style="font-family:verdana">The dates are stored as YYYY-MM-DD, the first 10 characters represent the date:

In [202]:
autos["date_crawled"].str[:10].value_counts(
                                            normalize=True, 
                                            dropna=False
                                            ).sort_values(ascending=False)

2016-03-07    0.041964
2016-03-14    0.038005
2016-04-03    0.037873
2016-03-08    0.037081
2016-03-21    0.036949
2016-03-17    0.036157
2016-03-15    0.036157
2016-03-09    0.036025
2016-03-20    0.035893
2016-03-30    0.035497
2016-03-25    0.035102
2016-03-31    0.034970
2016-03-22    0.034838
2016-03-23    0.033650
2016-03-12    0.033518
2016-03-29    0.033518
2016-03-28    0.033386
2016-04-02    0.033386
2016-04-04    0.033254
2016-03-19    0.032858
2016-03-16    0.032198
2016-03-11    0.031539
2016-04-01    0.030879
2016-03-24    0.030087
2016-03-26    0.029031
2016-03-27    0.028504
2016-03-10    0.026920
2016-03-05    0.022829
2016-03-06    0.015176
2016-03-18    0.013724
2016-04-05    0.012800
2016-03-13    0.012668
2016-04-06    0.002639
2016-04-07    0.000924
Name: date_crawled, dtype: float64

<span style="font-family:verdana">All the entries are from March and April 2016. The site was probably crawled around that time. The number of listings on each day are roughly uniform.</span>

### Exploring Price by Brand

<span style="font-family:verdana">Let's see what brands we have in this database:</span>

In [203]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.206255
opel              0.179203
ford              0.115730
renault           0.089074
bmw               0.047902
fiat              0.046582
audi              0.041700
peugeot           0.039720
mercedes_benz     0.037873
seat              0.023093
nissan            0.022829
mazda             0.020058
citroen           0.014516
mitsubishi        0.013328
volvo             0.010293
honda             0.009765
alfa_romeo        0.008314
suzuki            0.006994
hyundai           0.006730
daihatsu          0.005806
skoda             0.005674
sonstige_autos    0.005410
kia               0.005278
toyota            0.005146
chrysler          0.004883
daewoo            0.004223
rover             0.004091
smart             0.004091
subaru            0.003827
saab              0.003167
lancia            0.002639
trabant           0.002111
chevrolet         0.001979
lada              0.000396
mini              0.000396
porsche           0.000396
jeep              0.000264
d

<span style="font-family:verdana">As we can see Volkswagen is the most popular brand. Note that a lot of brands do not even account for 1% of the listings. Since they do not affect our analysis much, we limit our analysis to brands that account for at least 1% of our total listings. Our aim is to find out the mean prices of popular brands.</span>

In [204]:
brand_perc = autos["brand"].value_counts(normalize=True)
popular = brand_perc[brand_perc>0.01].index

brand_mean_prices = {}
for brand in popular:
    rows = autos[autos["brand"]==brand]
    mean_price = rows["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
brand_mean_prices

{'volkswagen': 604,
 'opel': 592,
 'ford': 573,
 'renault': 604,
 'bmw': 609,
 'fiat': 566,
 'audi': 638,
 'peugeot': 605,
 'mercedes_benz': 686,
 'seat': 617,
 'nissan': 626,
 'mazda': 587,
 'citroen': 657,
 'mitsubishi': 587,
 'volvo': 664}

<span style="font-family:verdana">Of these brands, a gradual price gap is noticed:</span>
- <span style="font-family:verdana"><b>audi, bmw, mercedez_benz</b> and <b>mini</b> are expensive,</span>
- <span style="font-family:verdana"><b>volkswagen, seat, skoda, mazda, nissan, toyota</b> and <b>hyundai</b> are mid-range,
- <span style="font-family:verdana"><b>opel, ford, renault, peugeot, fiat, smart</b> and <b>citroen</b> are less expensive.

<span style="font-family:verdana">Comparing this result to the popularity, volkswagen is the most popular probably due to its performance over its price. However, just this much is not enough to conclude that mid range car are popular in general. We notice that the expensive cars are more popular than less expensive ones. The trend followed is:

- expensive -> less expensive -> mid range.

### Exploring Mileage

<span style="font-family:verdana">Do not forget that mileage is recorded in the <i>odometer_km</i> column.

In [205]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns=["mean_prices"])

,mean_prices
volkswagen,604
opel,592
ford,573
renault,604
bmw,609
fiat,566
audi,638
peugeot,605
mercedes_benz,686
seat,617


In [206]:
brand_mean_mileage = {}
for brand in popular:
    rows = autos[autos["brand"]==brand]
    mean_mileage = rows["odometer_km"].fillna(np.nan).mean()
    if np.isfinite(mean_mileage):
        brand_mean_mileage[brand] = int(mean_mileage)
    else:
        brand_mean_mileage[brand] = 0

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)
popularity = pd.Series(autos["brand"].value_counts(normalize=True).sort_values(ascending=False))

In [207]:
brand_info = pd.DataFrame(mean_mileage, columns=["mean_mileage"])
brand_info["popularity"] = popularity
brand_info["mean_prices"] = mean_prices
brand_info

,mean_mileage,popularity,mean_prices
volkswagen,0,0.206255,604
opel,0,0.179203,592
ford,0,0.115730,573
renault,0,0.089074,604
bmw,0,0.047902,609
fiat,0,0.046582,566
audi,0,0.041700,638
peugeot,0,0.039720,605
mercedes_benz,0,0.037873,686
seat,0,0.023093,617


<span style="font-family:verdana">In general, expensive cars have higher mileage. We can see that the popularity of a car roughly depends on the mileage.</span>

## {Step 4} Conclusion

In [208]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7578 entries, 6 to 49989
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date_crawled        7578 non-null   object 
 1   name                7578 non-null   object 
 2   price               7578 non-null   float64
 3   abtest              7578 non-null   object 
 4   vehicle_type        6897 non-null   object 
 5   registration_year   7578 non-null   int64  
 6   gearbox             7274 non-null   object 
 7   power_ps            7578 non-null   int64  
 8   model               7133 non-null   object 
 9   odometer_km         7578 non-null   object 
 10  registration_month  7578 non-null   int64  
 11  fuel_type           6826 non-null   object 
 12  brand               7578 non-null   object 
 13  unrepaired_damage   5626 non-null   object 
 14  ad_created          7578 non-null   object 
 15  postal_code         7578 non-null   int64  
 16  last_

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,300.0,test,limousine,1995,manuell,90,golf,150000,8,benzin,volkswagen,NaN,2016-03-20 00:00:00,31535,2016-03-23 02:48:59
8,2016-03-22 16:51:34,Seat_Arosa,250.0,test,NaN,2000,manuell,0,arosa,150000,10,NaN,seat,nein,2016-03-22 00:00:00,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,590.0,control,bus,1997,manuell,90,megane,150000,7,benzin,renault,nein,2016-03-16 00:00:00,15749,2016-04-06 10:46:35
16,2016-03-16 14:59:02,Opel_Vectra_B_Kombi,350.0,test,kombi,1999,manuell,101,vectra,150000,5,benzin,opel,nein,2016-03-16 00:00:00,57299,2016-03-18 05:29:37
25,2016-03-21 21:56:18,Ford_escort_kombi_an_bastler_mit_ghia_ausstattung,90.0,control,kombi,1996,manuell,116,NaN,150000,4,benzin,ford,ja,2016-03-21 00:00:00,27574,2016-04-01 05:16:49


<span style="font-family:verdana">In conclusion we cleaned ~60.000 records which included:
- <span style="font-family:verdana">Wrong dates,
- <span style="font-family:verdana">null's,
- <span style="font-family:verdana">and another records which included the wrong data to analysis.

<span style="font-family:verdana">Also we deleted 3 columns and changed memory usage from 56.7+ MB to 43.5+ MB - what would be useful for convenient work with this database.